In [ ]:
%pip install finvizfinance

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from time import sleep
from random import uniform

def get_finviz_insider_trading(ticker, delay=True):
    """
    Scrape insider trading data from Finviz for a given ticker
    
    Args:
        ticker (str): Stock symbol (e.g., 'AAPL')
        delay (bool): Whether to add a random delay between requests
    
    Returns:
        pandas.DataFrame: Insider trading data
    """
    
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    }
    
    url = f"https://finviz.com/quote.ashx?t={ticker.upper()}"
    
    if delay:
        sleep(uniform(1, 3))
    
    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status() 
        
        soup = BeautifulSoup(response.text, 'html.parser')
        
        insider_table = soup.find('table', class_='body-table')
        
        if not insider_table:
            print(f"No insider trading data found for {ticker}")
            return None
        
        headers = []
        for th in insider_table.find('tr').find_all('th'):
            headers.append(th.text.strip())
        
        rows = []
        for tr in insider_table.find_all('tr')[1:]:
            row = [td.text.strip() for td in tr.find_all('td')]
            if len(row) == len(headers):
                rows.append(row)
        
        df = pd.DataFrame(rows, columns=headers)
        
        df['Ticker'] = ticker.upper()
        
        return df
    
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data for {ticker}: {e}")
        return None

if __name__ == "__main__":
    tickers = ['AAPL', 'MSFT', 'AMZN', 'TSLA', 'GOOGL']
    
    all_insider_data = []
    for ticker in tickers:
        print(f"Fetching insider data for {ticker}...")
        insider_data = get_finviz_insider_trading(ticker)
        if insider_data is not None:
            all_insider_data.append(insider_data)
        sleep(uniform(2, 4))
    
    if all_insider_data:
        combined_df = pd.concat(all_insider_data, ignore_index=True)
        print("\nCombined Insider Trading Data:")
        print(combined_df)
        
        combined_df.to_csv('finviz_insider_trading.csv', index=False)
        print("\nData saved to 'finviz_insider_trading.csv'")
    else:
        print("No data was retrieved.")

Fetching insider data for AAPL...
Fetching insider data for MSFT...
Fetching insider data for AMZN...
Fetching insider data for TSLA...
Fetching insider data for GOOGL...

Combined Insider Trading Data:
                    Insider Trading                  Relationship        Date  \
0                WILLIAMS JEFFREY E                           COO  Apr 02 '25   
1                    COOK TIMOTHY D       Chief Executive Officer  Apr 02 '25   
2                Adams Katherine L.         SVP, GC and Secretary  Apr 02 '25   
3                 KATHERINE L ADAMS                       Officer  Apr 02 '25   
4                WILLIAMS JEFFREY E                       Officer  Apr 02 '25   
..                              ...                           ...         ...   
323        PICHAI FAMILY FOUNDATION                      Director  Sep 04 '24   
324                   SUNDAR PICHAI                      Director  Sep 04 '24   
325            O'Toole Amie Thuener  VP, Chief Accounting Officer  S